# Forecasting Airline Stock Volatility with Oil Futures Volatility and Oil News Sentiment
## CS 329E — Phase 2: Data Analysis
**Group 23** | Blake Stanley, Shivsagar Palla, Raghuvendra Chowdhry

**Date:** February 27, 2026

## Step 0: Project Goal

The goal of this project is to investigate whether volatility in crude oil futures — measured by the CBOE Crude Oil Volatility Index (OVX) — and oil market news sentiment — captured by the Text Oil Sentiment Indicator (TOSI) — can predict stock price volatility for major U.S. airlines (AAL, DAL, UAL, LUV) and the JETS ETF (a global airline industry index). We hypothesize that increases in OVX and negative shifts in TOSI lead to increased airline stock volatility, with the strongest predictive power at a 1-month lag. Combining OVX with TOSI should yield a more accurate model than either alone.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

---
## Step 1: Familiarize with the Datasets
### 1.1 Load All Datasets

In [2]:
aal = pd.read_excel('data/AAL_IV.xlsx', index_col=0, parse_dates=True, skiprows=6)
aal.columns = ['Price', 'AAL_Volatility']
aal.drop(columns='Price', inplace=True)
aal.index.name = 'Date'
aal.head()

,AAL_Volatility
Date,
2026-02-27,39.9830
2026-02-26,40.9690
2026-02-25,40.1460
2026-02-24,39.7440
2026-02-23,40.4530


In [3]:
aal.tail()

,AAL_Volatility
Date,
2013-12-13,31.8910
2013-12-12,31.6640
2013-12-11,31.0870
2013-12-10,31.2680
2013-12-09,31.1840


In [4]:
# Load in Bloomberg datasets on airline volatility 
aal = pd.read_excel('data/AAL_IV.xlsx', index_col=0, parse_dates=True, skiprows=6)
aal.columns = ['Price', 'AAL_Volatility']
aal.drop(columns='Price', inplace=True)
aal.index.name = 'Date'

dal = pd.read_excel('data/DAL_IV.xlsx', index_col=0, parse_dates=True, skiprows=6)
dal.columns = ['Price', 'DAL_Volatility']
dal.drop(columns='Price', inplace=True)
dal.index.name = 'Date'

ual = pd.read_excel('data/UAL_IV.xlsx', index_col=0, parse_dates=True, skiprows=6)
ual.columns = ['Price', 'UAL_Volatility']
ual.drop(columns='Price', inplace=True)
ual.index.name = 'Date'

luv = pd.read_excel('data/LUV_IV.xlsx', index_col=0, parse_dates=True, skiprows=6)
luv.columns = ['Price', 'LUV_Volatility']
luv.drop(columns='Price', inplace=True)
luv.index.name = 'Date'

jets = pd.read_excel('data/JETS_IV.xlsx', index_col=0, parse_dates=True, skiprows=6)
jets.columns = ['Price', 'JETS_Volatility']
jets.drop(columns='Price', inplace=True)
jets.index.name = 'Date'

# Load OVX
ovx = pd.read_csv('data/OVXCLS.csv', parse_dates=['observation_date'])
ovx.columns = ['Date', 'OVX']
ovx['OVX'] = pd.to_numeric(ovx['OVX'], errors='coerce')
ovx = ovx.set_index('Date')

# Load TOSI
tosi = pd.read_csv('data/TOSI.csv')
tosi = tosi.iloc[:, :2]  # Keep only first two columns
tosi.columns = ['Date', 'TOSI']
tosi['Date'] = pd.to_datetime(tosi['Date'], format='%b-%y')
tosi = tosi.set_index('Date')

print('All datasets loaded successfully.')

All datasets loaded successfully.


### 1.2 Dataset Sizes (Rows × Columns)

In [5]:
datasets = {
    'AAL_IV': aal,
    'DAL_IV': dal,
    'UAL_IV': ual,
    'LUV_IV': luv,
    'JETS_IV': jets,
    'OVX': ovx,
    'TOSI': tosi
}

size_info = []
for name, df in datasets.items():
    size_info.append({
        'Dataset': name,
        'Rows': df.shape[0],
        'Columns': df.shape[1],
        'Column Names': ', '.join(df.columns),
        'Date Range': f'{df.index.min().strftime("%Y-%m-%d")} to {df.index.max().strftime("%Y-%m-%d")}'
    })

pd.DataFrame(size_info)

,Dataset,Rows,Columns,Column Names,Date Range
0,AAL_IV,3089,1,AAL_Volatility,2013-12-09 to 2026-02-27
1,DAL_IV,4738,1,DAL_Volatility,2007-05-01 to 2026-02-27
2,UAL_IV,3875,1,UAL_Volatility,2010-10-01 to 2026-02-27
3,LUV_IV,11511,1,LUV_Volatility,1980-07-28 to 2026-02-27
4,JETS_IV,2724,1,JETS_Volatility,2015-04-30 to 2026-02-27
5,OVX,4905,1,OVX,2007-05-10 to 2026-02-25
6,TOSI,526,1,TOSI,1982-01-01 to 2025-10-01


### 1.3 Column Classification

All datasets are time series with a **Date** index (ordinal) and a single **continuous** numeric column:

| Dataset | Column | Type | Units |
|---------|--------|------|-------|
| AAL_IV | AAL_Volatility | Continuous | Annualized volatility (implied volatility %) |
| DAL_IV | DAL_Volatility | Continuous | Annualized volatility (implied volatility %) |
| UAL_IV | UAL_Volatility | Continuous | Annualized volatility (implied volatility %) |
| LUV_IV | LUV_Volatility | Continuous | Annualized volatility (implied volatility %) |
| JETS_IV | JETS_Volatility | Continuous | Annualized volatility (implied volatility %) |
| OVX | OVX | Continuous | Index points (implied volatility %) |
| TOSI | TOSI | Continuous | Sentiment index (normalized, mean=100 in original) |

There are **no categorical or discrete variables** in any of the datasets. All columns are continuous quantitative data.

### 1.4 Descriptive Statistics for Quantitative Data

In [6]:
all_stats = []
for name, df in datasets.items():
    col = df.columns[0]
    s = df[col].dropna()
    all_stats.append({
        'Dataset': name,
        'Column': col,
        'Count': len(s),
        'Min': s.min(),
        'Max': s.max(),
        'Range': s.max() - s.min(),
        'Median': s.median(),
        'Mean': s.mean(),
        'Std Dev': s.std()
    })

stats_df = pd.DataFrame(all_stats)
stats_df

,Dataset,Column,Count,Min,Max,Range,Median,Mean,Std Dev
0,AAL_IV,AAL_Volatility,3076,12.4080,120.0760,107.6680,36.3360,39.5460,13.0874
1,DAL_IV,DAL_Volatility,4732,19.1320,278.8310,259.6990,39.4890,47.7141,26.9727
2,UAL_IV,UAL_Volatility,3874,20.3210,310.1330,289.8120,41.3755,44.4188,17.7364
3,LUV_IV,LUV_Volatility,7978,9.9160,205.7640,195.8480,35.1475,37.0856,12.1036
4,JETS_IV,JETS_Volatility,2707,10.9460,207.4440,196.4980,28.4750,32.6623,15.4367
5,OVX,OVX,4731,14.5000,325.1500,310.6500,35.1400,38.4096,16.9437
6,TOSI,TOSI,526,-4.0751,2.0867,6.1618,0.1220,-0.0000,0.9458


---
## Step 2: Data Wrangling
### 2.1 Missing Data Detection

In [7]:
print('=== Missing Values per Dataset ===')
for name, df in datasets.items():
    missing = df.isnull().sum()
    total = len(df)
    print(f'\n{name}:')
    for col in df.columns:
        n_miss = df[col].isnull().sum()
        print(f'  {col}: {n_miss} missing ({n_miss/total*100:.2f}%)')

=== Missing Values per Dataset ===

AAL_IV:
  AAL_Volatility: 13 missing (0.42%)

DAL_IV:
  DAL_Volatility: 6 missing (0.13%)

UAL_IV:
  UAL_Volatility: 1 missing (0.03%)

LUV_IV:
  LUV_Volatility: 3533 missing (30.69%)

JETS_IV:
  JETS_Volatility: 17 missing (0.62%)

OVX:
  OVX: 174 missing (3.55%)

TOSI:
  TOSI: 0 missing (0.00%)


In [8]:
# Show specific missing AAL rows
print('AAL Missing Rows:')
missing_rows = aal[aal.isnull().any(axis=1)]
print(missing_rows)

AAL Missing Rows:
            AAL_Volatility
Date                      
2023-11-02             NaN
2023-11-01             NaN
2022-10-20             NaN
2022-10-19             NaN
2022-10-18             NaN
2022-10-17             NaN
2022-10-14             NaN
2022-10-13             NaN
2022-10-12             NaN
2022-09-27             NaN
2022-09-26             NaN
2022-09-01             NaN
2021-06-07             NaN


In [9]:
# Show specific missing DAL rows
print('DAL Missing Rows:')
missing_rows = dal[dal.isnull().any(axis=1)]
print(missing_rows)

DAL Missing Rows:
            DAL_Volatility
Date                      
2026-02-27             NaN
2025-07-28             NaN
2008-03-28             NaN
2007-05-03             NaN
2007-05-02             NaN
2007-05-01             NaN


In [10]:
# Show specific missing UAL rows
print('UAL Missing Rows:')
missing_rows = ual[ual.isnull().any(axis=1)]
print(missing_rows)

UAL Missing Rows:
            UAL_Volatility
Date                      
2026-02-27             NaN


In [11]:
# Show specific missing LUV rows
print('LUV Missing Rows:')
missing_rows = luv[luv.isnull().any(axis=1)]
print(missing_rows)

LUV Missing Rows:
            LUV_Volatility
Date                      
2026-02-27             NaN
2006-11-03             NaN
2005-10-04             NaN
2005-09-16             NaN
2005-07-20             NaN
...                    ...
1980-08-01             NaN
1980-07-31             NaN
1980-07-30             NaN
1980-07-29             NaN
1980-07-28             NaN

[3533 rows x 1 columns]


In [12]:
# Show specific missing JETS rows
print('JETS Missing Rows:')
missing_rows = jets[jets.isnull().any(axis=1)]
print(missing_rows)

JETS Missing Rows:
            JETS_Volatility
Date                       
2026-02-27              NaN
2020-12-11              NaN
2016-05-05              NaN
2016-03-14              NaN
2016-03-08              NaN
2016-01-19              NaN
2015-05-14              NaN
2015-05-13              NaN
2015-05-12              NaN
2015-05-11              NaN
2015-05-08              NaN
2015-05-07              NaN
2015-05-06              NaN
2015-05-05              NaN
2015-05-04              NaN
2015-05-01              NaN
2015-04-30              NaN


In [13]:
# Show specific missing OVX rows
print('OVX missing value dates:')
print(ovx[ovx['OVX'].isnull()].index.tolist())

OVX missing value dates:
[Timestamp('2007-05-28 00:00:00'), Timestamp('2007-07-04 00:00:00'), Timestamp('2007-09-03 00:00:00'), Timestamp('2007-11-22 00:00:00'), Timestamp('2007-12-25 00:00:00'), Timestamp('2008-01-01 00:00:00'), Timestamp('2008-01-21 00:00:00'), Timestamp('2008-02-18 00:00:00'), Timestamp('2008-03-21 00:00:00'), Timestamp('2008-05-26 00:00:00'), Timestamp('2008-07-04 00:00:00'), Timestamp('2008-09-01 00:00:00'), Timestamp('2008-11-27 00:00:00'), Timestamp('2008-12-25 00:00:00'), Timestamp('2009-01-01 00:00:00'), Timestamp('2009-01-19 00:00:00'), Timestamp('2009-02-16 00:00:00'), Timestamp('2009-04-10 00:00:00'), Timestamp('2009-05-25 00:00:00'), Timestamp('2009-07-03 00:00:00'), Timestamp('2009-09-07 00:00:00'), Timestamp('2009-11-26 00:00:00'), Timestamp('2009-12-25 00:00:00'), Timestamp('2010-01-01 00:00:00'), Timestamp('2010-01-18 00:00:00'), Timestamp('2010-02-15 00:00:00'), Timestamp('2010-04-02 00:00:00'), Timestamp('2010-05-31 00:00:00'), Timestamp('2010-07-05 

### 2.2 Handling Missing Data

**AAL, DAL, UAL, JETS*:** Missing values correspond to days without enough call option volume for implied volatility to be calculated. Since these are not random gaps, we can impute the missing data with back fill to fix these small gaps. However, there are some gaps at the start of the dataset (the period after the initial stock/etf listing) where there was not enough volume for IV data, and we cannot front fill this (as it would be eaking future information into past observations, which is a big problem for a prediction model). For these gaps, we will remove them. (Note that datasets are in recent to old order, so backfill is using past data to impute future data.)

**LUV**: We have the same situation with LUV (and will back fill and drop accordingly), except that the start of the dataset has a very long period of no options data. This is because the dataset starts in the 1980s and there was much lower option availability and volume. The process is the same there just will be a larger quantity of rows lost due to this many year stretch of no data.

**OVX:** Missing values correspond to market holidays (e.g., Memorial Day, Independence Day, etc.) when the CBOE was closed. Since these are not random gaps but known non-trading days, we **remove these rows** rather than impute. Imputing with the mean would distort the time series, and these dates have no corresponding airline trading data either.

**Airline volatility and TOSI:** No missing values detected.

In [14]:
# Back fill small gaps in airline stock datasets
aal.bfill(inplace=True)
dal.bfill(inplace=True)
ual.bfill(inplace=True)
luv.bfill(inplace=True)
jets.bfill(inplace=True)

# Drop missing begging rows of airline stock datasets
aal.dropna(inplace=True)
dal.dropna(inplace=True)
ual.dropna(inplace=True)
luv.dropna(inplace=True)
jets.dropna(inplace=True)

In [15]:
# Drop missing OVX rows (market holidays)
ovx_before = len(ovx)
ovx = ovx.dropna()
ovx_after = len(ovx)
print(f'OVX: removed {ovx_before - ovx_after} rows with missing values (market holidays).')
print(f'OVX rows remaining: {ovx_after}')

# Update the datasets dict
datasets['OVX'] = ovx

OVX: removed 174 rows with missing values (market holidays).
OVX rows remaining: 4731


In [16]:
print('=== Verify No Missing Values per Dataset ===')
for name, df in datasets.items():
    missing = df.isnull().sum()
    total = len(df)
    print(f'\n{name}:')
    for col in df.columns:
        n_miss = df[col].isnull().sum()
        print(f'  {col}: {n_miss} missing ({n_miss/total*100:.2f}%)')

# print if any missing values remain in any dataset
any_missing = False
for name, df in datasets.items():
    if df.isnull().any().any():
        any_missing = True
        print(f'{name} still has missing values.')
if not any_missing:
    print('No missing values remain in any dataset.')

=== Verify No Missing Values per Dataset ===

AAL_IV:
  AAL_Volatility: 0 missing (0.00%)

DAL_IV:
  DAL_Volatility: 0 missing (0.00%)

UAL_IV:
  UAL_Volatility: 0 missing (0.00%)

LUV_IV:
  LUV_Volatility: 0 missing (0.00%)

JETS_IV:
  JETS_Volatility: 0 missing (0.00%)

OVX:
  OVX: 0 missing (0.00%)

TOSI:
  TOSI: 0 missing (0.00%)
No missing values remain in any dataset.


### 2.3 Range Validation

In [17]:
print('=== Range Validation ===')

# Airline volatility should be non-negative
for name in ['AAL_IV', 'DAL_IV', 'UAL_IV', 'LUV_IV', 'JETS_IV']:
    df = datasets[name]
    col = df.columns[0]
    neg = (df[col] < 0).sum()
    print(f'{name}: {neg} negative values (volatility must be >= 0)')

# OVX should be non-negative
neg_ovx = (ovx['OVX'] < 0).sum()
print(f'OVX: {neg_ovx} negative values (volatility index must be >= 0)')

# TOSI can be any real number (sentiment), so no range constraint
print(f'TOSI: range [{tosi["TOSI"].min():.4f}, {tosi["TOSI"].max():.4f}] — no constraint (sentiment index)')

print('\nAll values within expected ranges. No corrections needed.')

=== Range Validation ===
AAL_IV: 0 negative values (volatility must be >= 0)
DAL_IV: 0 negative values (volatility must be >= 0)
UAL_IV: 0 negative values (volatility must be >= 0)
LUV_IV: 0 negative values (volatility must be >= 0)
JETS_IV: 0 negative values (volatility must be >= 0)
OVX: 0 negative values (volatility index must be >= 0)
TOSI: range [-4.0751, 2.0867] — no constraint (sentiment index)

All values within expected ranges. No corrections needed.


### 2.4 Format Validation

In [18]:
print('=== Data Types ===')
for name, df in datasets.items():
    print(f'\n{name}:')
    print(f'  Index type: {type(df.index).__name__} (dtype: {df.index.dtype})')
    for col in df.columns:
        print(f'  {col}: {df[col].dtype}')

print('\nAll indices are DatetimeIndex and all value columns are float64. Formats are correct.')

=== Data Types ===

AAL_IV:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  AAL_Volatility: float64

DAL_IV:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  DAL_Volatility: float64

UAL_IV:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  UAL_Volatility: float64

LUV_IV:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  LUV_Volatility: float64

JETS_IV:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  JETS_Volatility: float64

OVX:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  OVX: float64

TOSI:
  Index type: DatetimeIndex (dtype: datetime64[ns])
  TOSI: float64

All indices are DatetimeIndex and all value columns are float64. Formats are correct.


### 2.5 Duplicate Detection

In [19]:
print('=== Duplicate Index (Date) Check ===')
for name, df in datasets.items():
    dups = df.index.duplicated().sum()
    print(f'{name}: {dups} duplicate dates')

print('\nNo duplicates found in any dataset.')

=== Duplicate Index (Date) Check ===
AAL_IV: 0 duplicate dates
DAL_IV: 0 duplicate dates
UAL_IV: 0 duplicate dates
LUV_IV: 0 duplicate dates
JETS_IV: 0 duplicate dates
OVX: 0 duplicate dates
TOSI: 0 duplicate dates

No duplicates found in any dataset.


### 2.6 Create Monthly Aggregated DataFrames

Since TOSI is a monthly indicator, we compute **monthly mean OVX** and **monthly mean volatility** for each airline to enable correlation analysis across all variables on a common time axis.

In [21]:
# Compute monthly means for OVX
ovx_monthly = ovx.resample('MS').mean()
ovx_monthly.columns = ['OVX_Monthly']

# Compute monthly means for each airline
aal_monthly = aal.resample('MS').mean()
aal_monthly.columns = ['AAL_Vol_Monthly']

dal_monthly = dal.resample('MS').mean()
dal_monthly.columns = ['DAL_Vol_Monthly']

ual_monthly = ual.resample('MS').mean()
ual_monthly.columns = ['UAL_Vol_Monthly']

luv_monthly = luv.resample('MS').mean()
luv_monthly.columns = ['LUV_Vol_Monthly']

jets_monthly = jets.resample('MS').mean()
jets_monthly.columns = ['JETS_Vol_Monthly']

# Merge all monthly data
monthly = ovx_monthly.join(tosi, how='outer')
monthly = monthly.join(aal_monthly, how='outer')
monthly = monthly.join(dal_monthly, how='outer')
monthly = monthly.join(ual_monthly, how='outer')
monthly = monthly.join(luv_monthly, how='outer')
monthly = monthly.join(jets_monthly, how='outer')

print(f'Monthly merged DataFrame shape: {monthly.shape}')
print(f'Date range: {monthly.index.min()} to {monthly.index.max()}')
print(f'\nMissing values per column:')
print(monthly.isnull().sum())
print(f'\nFirst 10 rows:')
monthly.head(10)

Monthly merged DataFrame shape: (530, 7)
Date range: 1982-01-01 00:00:00 to 2026-02-01 00:00:00

Missing values per column:
OVX_Monthly         304
TOSI                  4
AAL_Vol_Monthly     383
DAL_Vol_Monthly     304
UAL_Vol_Monthly     345
LUV_Vol_Monthly     146
JETS_Vol_Monthly    400
dtype: int64

First 10 rows:


,OVX_Monthly,TOSI,AAL_Vol_Monthly,DAL_Vol_Monthly,UAL_Vol_Monthly,LUV_Vol_Monthly,JETS_Vol_Monthly
Date,,,,,,,
1982-01-01,NaN,0.6657,NaN,NaN,NaN,NaN,NaN
1982-02-01,NaN,-0.3537,NaN,NaN,NaN,NaN,NaN
1982-03-01,NaN,-0.4060,NaN,NaN,NaN,NaN,NaN
1982-04-01,NaN,0.5299,NaN,NaN,NaN,NaN,NaN
1982-05-01,NaN,0.4202,NaN,NaN,NaN,NaN,NaN
1982-06-01,NaN,0.0997,NaN,NaN,NaN,NaN,NaN
1982-07-01,NaN,-0.5897,NaN,NaN,NaN,NaN,NaN
1982-08-01,NaN,-0.0477,NaN,NaN,NaN,NaN,NaN
1982-09-01,NaN,0.3981,NaN,NaN,NaN,NaN,NaN


In [22]:
# Create a common-period dataframe (all columns non-null) for correlation
monthly_complete = monthly.dropna()
print(f'Monthly complete (no NaN) DataFrame shape: {monthly_complete.shape}')
print(f'Date range: {monthly_complete.index.min()} to {monthly_complete.index.max()}')
monthly_complete.describe()

Monthly complete (no NaN) DataFrame shape: (126, 7)
Date range: 2015-05-01 00:00:00 to 2025-10-01 00:00:00


,OVX_Monthly,TOSI,AAL_Vol_Monthly,DAL_Vol_Monthly,UAL_Vol_Monthly,LUV_Vol_Monthly,JETS_Vol_Monthly
count,126.0000,126.0000,126.0000,126.0000,126.0000,126.0000,126.0000
mean,39.8175,-0.6790,40.5322,37.9718,44.2276,34.7888,32.6673
std,16.8714,0.9708,13.2479,15.3138,19.5811,11.7250,14.1645
min,21.5870,-4.0751,25.6983,22.2460,22.1464,19.7790,15.6993
25%,31.6136,-1.2788,33.9236,28.7061,32.8607,28.0204,24.9900
50%,37.0891,-0.6532,36.6334,34.3742,39.7439,32.8496,28.6520
75%,43.9267,-0.0474,41.0891,40.6682,47.2478,37.2790,34.8151
max,163.6524,1.3338,106.4759,125.8943,160.9412,105.1844,114.5857


---
## Step 3: Preliminary Statistical Analysis
### 3.1 Pearson Correlation Coefficient Matrix

In [23]:
# Full correlation matrix on the complete monthly data
corr_matrix = monthly_complete.corr(method='pearson')
print('Pearson Correlation Matrix (monthly data, overlapping period):')
corr_matrix

Pearson Correlation Matrix (monthly data, overlapping period):


,OVX_Monthly,TOSI,AAL_Vol_Monthly,DAL_Vol_Monthly,UAL_Vol_Monthly,LUV_Vol_Monthly,JETS_Vol_Monthly
OVX_Monthly,1.0000,-0.5911,0.5009,0.7928,0.7679,0.7731,0.8110
TOSI,-0.5911,1.0000,-0.3850,-0.4746,-0.4450,-0.4573,-0.4609
AAL_Vol_Monthly,0.5009,-0.3850,1.0000,0.2605,0.2323,0.2402,0.3576
DAL_Vol_Monthly,0.7928,-0.4746,0.2605,1.0000,0.9933,0.9744,0.9539
UAL_Vol_Monthly,0.7679,-0.4450,0.2323,0.9933,1.0000,0.9772,0.9456
LUV_Vol_Monthly,0.7731,-0.4573,0.2402,0.9744,0.9772,1.0000,0.9364
JETS_Vol_Monthly,0.8110,-0.4609,0.3576,0.9539,0.9456,0.9364,1.0000


In [24]:
# Also compute pairwise correlations maximizing available data for each pair
print('Pairwise Pearson correlations (maximizing data for each pair):\n')
cols = monthly.columns.tolist()
pairwise_results = []
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        pair = monthly[[cols[i], cols[j]]].dropna()
        if len(pair) > 2:
            r = pair.corr().iloc[0, 1]
            pairwise_results.append({
                'Variable 1': cols[i],
                'Variable 2': cols[j],
                'Pearson r': r,
                'N (months)': len(pair),
                '|r|': abs(r)
            })

pairwise_df = pd.DataFrame(pairwise_results).sort_values('|r|', ascending=False)
pairwise_df

Pairwise Pearson correlations (maximizing data for each pair):



,Variable 1,Variable 2,Pearson r,N (months),|r|
15,DAL_Vol_Monthly,UAL_Vol_Monthly,0.9667,185,0.9667
17,DAL_Vol_Monthly,JETS_Vol_Monthly,0.9512,130,0.9512
19,UAL_Vol_Monthly,JETS_Vol_Monthly,0.9443,130,0.9443
18,UAL_Vol_Monthly,LUV_Vol_Monthly,0.9363,185,0.9363
20,LUV_Vol_Monthly,JETS_Vol_Monthly,0.9312,130,0.9312
5,OVX_Monthly,JETS_Vol_Monthly,0.8083,130,0.8083
4,OVX_Monthly,LUV_Vol_Monthly,0.8013,226,0.8013
16,DAL_Vol_Monthly,LUV_Vol_Monthly,0.7864,226,0.7864
3,OVX_Monthly,UAL_Vol_Monthly,0.7161,185,0.7161
0,OVX_Monthly,TOSI,-0.6093,222,0.6093


### 3.2 Best Fit for Highly Correlated Pairs

We identify pairs with |r| > 0.6 and test linear, quadratic, and cubic fits.

In [25]:
from numpy.polynomial import polynomial as P

# Identify highly correlated pairs (|r| > 0.6)
high_corr = pairwise_df[pairwise_df['|r|'] > 0.6].copy()
print(f'Pairs with |r| > 0.6: {len(high_corr)}')
print()

fit_results = []
for _, row in high_corr.iterrows():
    v1, v2 = row['Variable 1'], row['Variable 2']
    pair = monthly[[v1, v2]].dropna()
    x = pair[v1].values
    y = pair[v2].values
    
    r2_scores = {}
    for degree, label in [(1, 'Linear'), (2, 'Quadratic'), (3, 'Cubic')]:
        coeffs = np.polyfit(x, y, degree)
        y_pred = np.polyval(coeffs, x)
        ss_res = np.sum((y - y_pred)**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        r2 = 1 - ss_res / ss_tot
        r2_scores[label] = r2
    
    best_fit = max(r2_scores, key=r2_scores.get)
    fit_results.append({
        'Variable 1': v1,
        'Variable 2': v2,
        'Pearson r': row['Pearson r'],
        'R² Linear': r2_scores['Linear'],
        'R² Quadratic': r2_scores['Quadratic'],
        'R² Cubic': r2_scores['Cubic'],
        'Best Fit': best_fit
    })

fit_df = pd.DataFrame(fit_results)
fit_df

Pairs with |r| > 0.6: 10



,Variable 1,Variable 2,Pearson r,R² Linear,R² Quadratic,R² Cubic,Best Fit
0,DAL_Vol_Monthly,UAL_Vol_Monthly,0.9667,0.9345,0.9447,0.9447,Cubic
1,DAL_Vol_Monthly,JETS_Vol_Monthly,0.9512,0.9048,0.9096,0.9116,Cubic
2,UAL_Vol_Monthly,JETS_Vol_Monthly,0.9443,0.8917,0.8960,0.9011,Cubic
3,UAL_Vol_Monthly,LUV_Vol_Monthly,0.9363,0.8766,0.8798,0.8823,Cubic
4,LUV_Vol_Monthly,JETS_Vol_Monthly,0.9312,0.8672,0.8746,0.8836,Cubic
5,OVX_Monthly,JETS_Vol_Monthly,0.8083,0.6534,0.6726,0.6984,Cubic
6,OVX_Monthly,LUV_Vol_Monthly,0.8013,0.6421,0.6552,0.6868,Cubic
7,DAL_Vol_Monthly,LUV_Vol_Monthly,0.7864,0.6185,0.6480,0.6660,Cubic
8,OVX_Monthly,UAL_Vol_Monthly,0.7161,0.5127,0.5186,0.5888,Cubic
9,OVX_Monthly,TOSI,-0.6093,0.3713,0.3801,0.3879,Cubic


### 3.3 Key Correlation Insights

**OVX vs. Airline Volatility:** We expect positive correlation — when oil uncertainty rises, airline stock volatility should increase.

**TOSI vs. Airline Volatility:** We expect negative correlation — positive oil sentiment (higher TOSI) should correspond to lower airline volatility.

**Airline Cross-correlations:** The airline stocks are likely highly correlated with each other since they share the same sector exposure.

In [26]:
# Focus on OVX and TOSI correlations with airline volatilities
print('=== Key Predictor-Target Correlations ===')
targets = ['AAL_Vol_Monthly', 'DAL_Vol_Monthly', 'UAL_Vol_Monthly', 'LUV_Vol_Monthly', 'JETS_Vol_Monthly']
predictors = ['OVX_Monthly', 'TOSI']

for pred in predictors:
    print(f'\n{pred} correlations:')
    for tgt in targets:
        pair = monthly[[pred, tgt]].dropna()
        if len(pair) > 2:
            r = pair.corr().iloc[0, 1]
            print(f'  vs {tgt}: r = {r:.4f}  (N = {len(pair)} months)')

=== Key Predictor-Target Correlations ===

OVX_Monthly correlations:
  vs AAL_Vol_Monthly: r = 0.5195  (N = 147 months)
  vs DAL_Vol_Monthly: r = 0.5674  (N = 226 months)
  vs UAL_Vol_Monthly: r = 0.7161  (N = 185 months)
  vs LUV_Vol_Monthly: r = 0.8013  (N = 226 months)
  vs JETS_Vol_Monthly: r = 0.8083  (N = 130 months)

TOSI correlations:
  vs AAL_Vol_Monthly: r = -0.3885  (N = 143 months)
  vs DAL_Vol_Monthly: r = -0.2650  (N = 222 months)
  vs UAL_Vol_Monthly: r = -0.4159  (N = 181 months)
  vs LUV_Vol_Monthly: r = -0.3737  (N = 380 months)
  vs JETS_Vol_Monthly: r = -0.4609  (N = 126 months)


---
## Step 5: Machine Learning Models (Outline)

### Model 1: MIDAS Regression (Mixed Data Sampling)

**Rationale:** Our predictors operate at different frequencies — OVX is daily, TOSI is monthly, and airline volatility is daily. MIDAS regression is specifically designed for mixed-frequency settings. It uses a weighting scheme (e.g., Beta or Almon lag polynomials) to aggregate the high-frequency OVX data to match the lower-frequency TOSI without losing information through simple averaging.

**Approach:**
- Dependent variable: Monthly airline volatility (or daily, depending on formulation)
- High-frequency predictor: Daily OVX, weighted via a Beta polynomial lag structure
- Low-frequency predictor: Monthly TOSI
- Test at multiple lag horizons: 1-month, 3-month, 6-month
- Evaluate with out-of-sample RMSE and directional accuracy
- Compare: MIDAS with OVX only, MIDAS with TOSI only, MIDAS with both

### Model 2: XGBoost / Random Forest

**Rationale:** Tree-based ensemble methods can capture non-linear relationships and interactions between OVX, TOSI, and airline volatility that linear models would miss. XGBoost in particular handles mixed feature types well and provides feature importance rankings.

**Approach:**
- Features: Lagged monthly OVX (mean, max, min, std of daily values), lagged TOSI, lagged airline volatility (autoregressive component)
- Target: Next-month airline volatility
- Train/test split: 80/20 temporal split (no shuffle — time series)
- Hyperparameter tuning via time-series cross-validation
- Evaluate with out-of-sample RMSE and compare to MIDAS baseline
- Feature importance analysis to identify which predictors drive volatility

---
## Step 4: Narration

Following the storytelling principles of Kosara and Mackinlay, our narration proceeds as follows:

- **Opening (Setting the Stage):** Airlines spend 25–35% of operating costs on fuel, making them uniquely vulnerable to oil market disruptions. When oil volatility spikes, airline stocks react — but how quickly, and can we predict it?

- **Introducing the Data:** We have three types of data: (1) daily airline stock volatility from GJR-GARCH models for AAL, DAL, UAL, LUV, and JETS; (2) the CBOE Oil Volatility Index (OVX), a daily measure of oil market uncertainty; and (3) TOSI, a monthly text-based oil sentiment indicator derived from NLP analysis of millions of news articles.

- **The Oil–Airline Connection:** Our correlation analysis reveals that OVX is positively correlated with airline volatility — when oil uncertainty rises, airline stocks become more volatile. TOSI shows the expected negative relationship — positive oil sentiment corresponds to calmer airline markets.

- **The Mixed-Frequency Challenge:** OVX moves daily, TOSI moves monthly, and we need to predict daily or monthly airline volatility. This mismatch motivates our use of MIDAS regression, which can optimally combine data at different frequencies.

- **The Prediction Question (Climax):** Can combining traditional volatility data (OVX) with cutting-edge NLP sentiment (TOSI) beat models using either alone? Our MIDAS and XGBoost models will test this at multiple lag horizons.

- **Expected Resolution:** We hypothesize that the combined model outperforms, especially during crisis periods (COVID-19, oil price wars) when sentiment data captures information faster than traditional volatility indices.

In [27]:
print('Analysis complete. All results are ready for the Phase 2 report.')

Analysis complete. All results are ready for the Phase 2 report.
